In [57]:
import tqdm
import pickle
import random
import numpy as np

from time import sleep
from collections import defaultdict
from grid import Grid, RandomGridGenerator

from IPython.display import clear_output
from utils import two_int_to_hash, numpy_arr_to_str, approximate_5x5_to_3x3

In [58]:
# GLOBAL CONFIGS
GRID_NUMBER_OF_ROWS = 100
GRID_NUMBER_OF_COLS = 100

PREDATOR_VISION_DIST = 3
PREY_VISION_DIST = 2

PREDATOR_VALUE = 1
PREY_VALUE = 2

PREDATOR_APPROXIMATE_SHARE = 0.2
PREY_APPROXIMATE_SHARE = 0.8

PREDATOR_DEFAULT_HEALTH = 100
PREY_DEFAULT_HEALTH = 60

alpha = 0.1
gamma = 0.6
epsilon = 0.1

action_space = [
    "up",
    "down",
    "left",
    "right",
    "stay",
]

[]

action_idxs = [0,1,2,3,4]


with open("DL_pretrained_model/predator_q_table.pickle", "rb") as f:
    pre_trained_data = pickle.load(f)


In [59]:
class AgentNet:
    def __init__(self):
        self.q_table = dict()

    def choose_next_cell(self, state):
        if state.shape[0] == 5:
            state = approximate_5x5_to_3x3(state)

        hashed_state = numpy_arr_to_str(state)

        if hashed_state not in self.q_table:
             self.q_table[hashed_state] = self._get_default_knowledge(hashed_state)

        allowed_actions = self._get_allowed_actions(state)

        if random.uniform(0, 1) < epsilon:
            return random.choice(allowed_actions)
        else:
            return np.argmax(self.q_table[hashed_state][allowed_actions,])

    def _get_allowed_actions(self, state):
        actions = [4]

        if state[0][1] == 0:
            actions.append(0)

        if state[2][1] == 0:
            actions.append(1)

        if state[1][0] == 0:
            actions.append(2)

        if state[1][2] == 0:
            actions.append(3)

        return actions

    def _get_default_knowledge(self, hashed_state):
        val = pre_trained_data[hashed_state]
        return np.array(
            [
            val[1],
            val[3],
            val[4],
            val[2],
            val[0],])




class PreyNet(AgentNet):
    pass

class PredatorNet(AgentNet):
    pass


In [60]:
predator_net = PredatorNet()
prey_net = PreyNet()

In [61]:
class Animal:
    __slots__ = "last_move", "remaining_life", "_type"
    def __init__(self, remaining_life, _type):
        self.last_move = None
        self.remaining_life = remaining_life
        self._type = _type

    def process_epoch(self):
        self.remaining_life -= 1

    def move(self, vision_matrix):
        if self._type == PREDATOR_VALUE:
            return predator_net.choose_next_cell(vision_matrix)
        elif self._type == PREY_VALUE:
            return prey_net.choose_next_cell(vision_matrix)


In [62]:
def get_dest_from_action(action, row, col):
    action = action_space[action]
    if action == "down":
        return (row+1) % GRID_NUMBER_OF_ROWS, col

    elif action == "up":
        return (row-1) % GRID_NUMBER_OF_ROWS, col

    elif action == "left":
        return row, (col-1)%GRID_NUMBER_OF_COLS

    elif action == "right":
        return row, (col+1)%GRID_NUMBER_OF_COLS

    elif action == "stay":
        return row, col

    else:
        raise Exception(f"Action can't be {action}")
    
def grid_of_nums_to_objects(grid: Grid) -> np.array:
    new_grid = np.empty(grid.grid.shape, dtype=object)
    for rowidx, row in enumerate(grid.grid):
        for colidx, val in enumerate(row):
            if val == PREDATOR_VALUE:     
                new_grid[rowidx][colidx] = Animal(PREDATOR_DEFAULT_HEALTH, PREDATOR_VALUE)
            elif val == PREY_VALUE:
                new_grid[rowidx][colidx] = Animal(PREY_DEFAULT_HEALTH, PREY_VALUE)

    return new_grid


In [63]:
rgg = RandomGridGenerator(GRID_NUMBER_OF_ROWS, GRID_NUMBER_OF_COLS)

generated_grid = rgg.getGrid(round(PREDATOR_APPROXIMATE_SHARE * 100) * [PREDATOR_VALUE] + round(PREY_APPROXIMATE_SHARE * 100) * [PREY_VALUE])

In [64]:
grid = Grid(generated_grid)
object_grid = grid_of_nums_to_objects(grid)



In [66]:


prey_c = []
pred_c = []


for time in tqdm.tqdm(range(10000)):

    next_grid = np.zeros((grid.rown, grid.coln))
    next_object_grid = np.empty(object_grid.shape, dtype=object)

    for row, line in enumerate(grid.grid):
        for col, item in enumerate(line):
            if item == PREY_VALUE:
                prey: Animal = object_grid[row][col]
                prey.process_epoch()
                if prey.remaining_life == 0:
                    grid.grid[row][col] = 0
                    object_grid[row][col] = None
                    continue

                if time % 4 == 0:
                    vision_space = grid.getNeighbors(row, col, PREY_VISION_DIST)
                    action = prey.move(vision_space)

                    dest = get_dest_from_action(action, row, col)
                    next_grid[dest[0], dest[1]] = item
                    next_object_grid[dest[0], dest[1]] = prey

                else:
                    next_grid[row, col] = item
                    next_object_grid[row, col] = prey

            if item == PREDATOR_VALUE:
                predator: Animal = object_grid[row][col]
                predator.process_epoch()
                if predator.remaining_life == 0:
                    grid.grid[row][col] = 0
                    object_grid[row][col] = None
                    continue

                if time % 4 == 0:
                    reward = 0
                    vision_space = grid.getNeighbors(row, col, PREDATOR_VISION_DIST)
                    action = predator.move(vision_space)

                    dest = get_dest_from_action(action, row, col)
                    if next_grid[dest[0], dest[1]] == PREY_VALUE:
                        reward = 1


                    app_next_vs = approximate_5x5_to_3x3(grid.getNeighborsFromNext(next_grid, dest[0], dest[1], PREDATOR_VISION_DIST))
                    app_vs = approximate_5x5_to_3x3(vision_space)

                    if predator_net.q_table.get(numpy_arr_to_str(app_next_vs)) is None:
                        predator_net.q_table[numpy_arr_to_str(app_next_vs)] = predator_net._get_default_knowledge(numpy_arr_to_str(app_next_vs))

                    max_value = np.max(predator_net.q_table[numpy_arr_to_str(app_next_vs)])
                    new_q_value = (1 - alpha) * predator_net.q_table[numpy_arr_to_str(app_vs)][action] + alpha * (reward + gamma * max_value)
                    predator_net.q_table[numpy_arr_to_str(app_vs)][action] = new_q_value



                    next_grid[dest[0], dest[1]] = item
                    next_object_grid[dest[0], dest[1]] = predator

                else:
                    next_grid[row, col] = item
                    next_object_grid[row, col] = predator

    grid = Grid(next_grid)
    object_grid = next_object_grid


    prey_count = np.count_nonzero(grid.grid == PREY_VALUE)
    pred_count = np.count_nonzero(grid.grid == PREDATOR_VALUE)
    prey_c.append(prey_count)
    pred_c.append(pred_count)

    if prey_count == 0 and pred_count == 0:
        print("no more animals on the grid")
        break;
        # for row, line in enumerate(grid.grid):
        #     for col, item in enumerate(line):
        #         if item == 0 and random.uniform(0,1) < 0.1:
        #             grid.grid[row][col] = PREY_VALUE

    # if time % 100 == 0:
    # clear_output(wait=True)
    # print("Episode: {}".format(time + 1))
    # print(grid.grid)
    # print(object_grid)


  0%|                                                                                       | 0/10000 [00:00<?, ?it/s]


KeyError: '20000|00000|00100|20000|00000'

In [ ]:
pred_c

[395,
 395,
 395,
 395,
 387,
 387,
 387,
 387,
 376,
 376,
 376,
 376,
 366,
 366,
 366,
 366,
 354,
 354,
 354,
 354,
 347,
 347,
 347,
 347,
 337,
 337,
 337,
 337,
 326,
 326,
 326,
 326,
 316,
 316,
 316,
 316,
 306,
 306,
 306,
 306,
 303,
 303,
 303,
 303,
 298,
 298,
 298,
 298,
 294,
 294,
 294,
 294,
 288,
 288,
 288,
 288,
 286,
 286,
 286,
 286,
 286,
 286,
 286,
 286,
 284,
 284,
 284,
 284,
 284,
 284,
 284,
 284,
 282,
 282,
 282,
 282,
 281,
 281,
 281,
 281,
 279,
 279,
 279,
 279,
 278,
 278,
 278,
 278,
 276,
 276,
 276,
 276,
 276,
 276,
 276,
 276,
 275,
 275,
 275,
 0]